# Exemple de notebook : Utilisation du package IBIS pour pré-filtrer, faire des jointures et aggréger avant de charger les données avec pandas

Avec la collecte de plusieurs années d’historique des données SISE, les volumes de données disponibles dans les tables edc_communes, edc_prelevements et edc_resultats vont significativement augmenter. Cette situationsituation pose des défis en termes de performances pour le chargement des données.

En particulier, une approche consistant à charger une table entière dans un DataFrame Pandas avec une commande comme :

```python
df = con.table("edc_resultats").df()
```

peut devenir lente et gourmande en mémoire.


Ibis permet de se connecter à la base de données et de la requêter via python.

On peut alors définir une suite d'opérations qui ne seront exécutées que lorsqu'on en fera la demande (via `.execute()` ou `.to_pandas()`). 
Une fois la requête exécuté, les données sont récupérées dans des DataFrames (ou Series) pandas.

```python
# Aucune donnée n'est chargée
table_filtree = table.filter(...)['colonnes']

# On execute la requête et on récupère les données dans un DataFrame pandas
df = table_filtree.execute()
```
    
Ce notebook a pour objectif de montrer :

- Comment utiliser [Ibis](https://ibis-project.org/tutorials/getting_started) pour interroger et filtrer efficacement les données avant de les charger dans Pandas.
- Des exemples concrets adaptés à vos besoins d'analyse, comme le filtrage par dates, jointures entre tables ou la sélection de paramètres spécifiques.

In [5]:
import pandas as pd

pd.set_option("display.max_columns", None)  # show all cols
pd.set_option("display.max_colwidth", None)  # show full width of showing cols
pd.set_option(
    "display.expand_frame_repr", False
)  # print cols side by side as it's supposed to be

In [7]:
# Connexion à la base de données via ibis
import ibis
from pipelines.tasks._common import DUCKDB_FILE

ibis_con = ibis.connect(DUCKDB_FILE, read_only=True)

Pour commencer, listons les tables disponibles dans la base de données :

In [ ]:
ibis_con.list_tables()

L'inspection des colonnes d'une table peut se faire directement avec Ibis.

In [ ]:
# On peut définir des objets DatabaseTable pour chacunes des tables
# Pour le moment on ne récupère pas les enregistrements des tables.
communes_table = ibis_con.table("edc_communes")
prelevements_table = ibis_con.table("edc_prelevements")
resultats_table = ibis_con.table("edc_resultats")

display(communes_table)
display(prelevements_table)
display(resultats_table)

## Filtres

1. Filtrer sur les prélévements de 2024

In [ ]:
# On prédéfinit une opération de filtre à effectuer sur la table
prelevements_2024 = prelevements_table.filter(
    prelevements_table.dateprel >= "2024-01-01"
)

# On peut mettre une limite avant d'éxecuter la requête.
df_prelevements_2024 = prelevements_2024.limit(50).execute()
print(type(df_prelevements_2024))
df_prelevements_2024.head(2)

2. Filtrer sur les prélévements non conformes en 2024



In [ ]:
non_conforme_query = (
    (prelevements_table.plvconformitebacterio == "N")
    | (prelevements_table.plvconformitechimique == "N")
    | (prelevements_table.plvconformitereferencebact == "N")
    | (prelevements_table.plvconformitereferencechim == "N")
)

non_conformes_2024 = prelevements_2024.filter(non_conforme_query)
f"En 2024, il y a eu {non_conformes_2024.count().execute():,.0f} prélévements non conformes au sens d'au moins une des variables suivantes : plvconformitebacterio, plvconformitechimique, plvconformitereferencebact, plvconformitereferencechim ([voir documentation](https://www.data.gouv.fr/fr/datasets/r/36afc708-42dc-4a89-b039-7fde6bcc83d8))".replace(
    ",", " "
)

In [ ]:
non_conformes_paris = non_conformes_2024.filter(
    (prelevements_table.nomcommuneprinc == "PARIS")
)
f"{non_conformes_paris.count().execute():,.0f} sur la ville de PARIS"

## Selectionner des colonnes avant d'exécuter la requête

In [ ]:
selected_columns = non_conformes_2024[
    ["referenceprel", "dateprel", "nomcommuneprinc", "plvconformitebacterio"]
]
selected_columns.execute()

## Jointure
Joindre edc_prelevements et edc_resultats sur referenceprel pour obtenir les résultats associés à chaque prélèvement :

In [ ]:
joined_data = non_conformes_2024.join(
    resultats_table, non_conformes_2024.referenceprel == resultats_table.referenceprel
)[
    [
        "referenceprel",
        "dateprel",
        "nomcommuneprinc",
        "libmajparametre",
        "insituana",
        "rqana",
        "cdunitereferencesiseeaux",
    ]
]

joined_data.execute()

## Groupby et aggregats
Nombre total de prélèvements non conforme par commune en 2024

In [ ]:
agg_data = non_conformes_2024.group_by("nomcommuneprinc").aggregate(
    nb_prelevements_non_conformes=non_conformes_2024.referenceprel.count()
)
agg_data.execute().sort_values("nb_prelevements_non_conformes", ascending=False)